# Data Collation Preperation Stage

## 1 - CASDA RM Catalog

From astropy table - Gaensler et. al. 2010.

In [4]:
import numpy as np
from astropy.io import ascii

rm_data = ascii.read("../data_catalog/CASDA_RMs.ecsv")

rm_data[0][0]

<SkyCoord (ICRS): (ra, dec) in deg
    (202.81578063, -16.12678334)>

Our data is already in the desired format.

## 2 - GASS HI Emissions

From Westmeier et. al. 2018 - in the form of a fits file.

In [40]:
from astropy.wcs import WCS
from astropy.io import fits
import matplotlib.pyplot as plt

hdu = fits.open("../data_catalog/hi4pi-hvc-nhi-ait.fits")
header = hdu[0].header
image = hdu[0].data

wcs = WCS(header)

#for i in range(im_shape[0]):
#    for j in range(im_shape[1]):
#        h1 = image[i][j]
#        coord = wcs.pixel_to_world(i, j).icrs
#       im_list.append([h1, coord])

coord = wcs.world_to_pixel(rm_data[0][0])
print(coord)
print(image[coord[0].astype(int)][coord[1].astype(int)])

(array(819.05876512), array(1593.25931717))
nan


## 3 - GASS HVC locations

From Moss et. al. 2013 - in the form of a VOTable

In [44]:
"""
Quick-and-dirty HVC
"""
from astropy.coordinates import SkyCoord
from astropy.coordinates import ICRS
from astropy.table import Table
import astropy.units as u
from astropy.io.votable import parse_single_table
import subprocess
import matplotlib.pyplot as plt
import numpy as np

"""
This script simply consists of steps to cobbled together. It is not to just be run start to finish
"""

"""
Filter Moss+2013 table to objects of a certain area
"""

def format_sexagesimal(coord):
    ra, dec = coord.split()[:3], coord.split()[3:]
    ra_formatted = ':'.join(ra)
    dec_formatted = f"{dec[0]}:{dec[1]}:{dec[2]}"
    return f"{ra_formatted} {dec_formatted}"

# Read the VOTable file
table = parse_single_table('../data_catalog/vizier_Moss2013_HVCs.vot').to_table()

#table

# Mask to decent sized area
#mask = (table['Area']>1) & (table['Area']<np.pi)

#table_big_HVCs = table[mask]

#table_big_HVCs

# Assuming the coordinates are in columns 'ra' and 'dec', adjust as needed
ra_dec = [f"{row['RAJ2000']} {row['DEJ2000']}" for row in table]#table_big_HVCs]
ra_dec = [format_sexagesimal(f"{row['RAJ2000']} {row['DEJ2000']}") for row in table]#table_big_HVCs]

ra_dec = SkyCoord(ra_dec,frame='fk5',unit=('hour','deg')).icrs

# Data is now in desired structure
table.add_column(ra_dec, index=1, name="SkyCoord")

table.info

#print(ICRS(ra=table["RAJ2000"][0], dec=table["DECJ2000"][0]))

<Table length=1693>
  name    dtype    unit   format                                    description                                       class    
-------- ------- ------- ------- --------------------------------------------------------------------------------- ------------
    Name   str15                                                                  HVC/AVC name GLLL.l+BB.b+VVV (1) MaskedColumn
SkyCoord  object deg,deg                                                                                               SkyCoord
 RAJ2000   str11                                                              Peak Hour of Right Ascension (J2000) MaskedColumn
 DEJ2000    str9                                                                Peak Degree of Declination (J2000) MaskedColumn
    VLSR float32  km / s {:6.1f}                                        [-483/481] Local Standard of Rest velocity MaskedColumn
  e_VLSR float32  km / s {:4.1f}                                                    

## 4 - H-alpha measurements

Finkbeiner et. al. 2003

In the form of a Healpy array

## 5 - Interpolated full collated data conversion

Hutschenreuter et. al. 2022;  Van Eck et. al. (in prep.)

Healpy is not supported on windows!

In [3]:
#import healpy as hp
import h5py

f = h5py.File('../data_catalog/faraday_sky_wo_ff_Hutschenreuter_2020.hdf5', 'r')

#imshow values, plot specific locations of interest. 
healpix_ringorder_N128 = f['faraday sky']['mean']
#healpix_ringorder_N128 = f['faraday sky']['std']
#hp.mollview(np.array(healpix_ringorder_N128), title="Mollview image",max=100,min=-100,rot=(-123,-56,0),cmap='seismic')
#hp.mollview(np.array(healpix_ringorder_N128), title="Mollview image",max=50,min=0,rot=(-123,-56,0),cmap='seismic')
#hp.graticule()

#cpos_src_obj_heal = SkyCoord('16h14m00.0s', '-60d35m00.0s', unit=(units.deg, units.deg), frame='icrs') 
#l = cpos_src_obj_heal.galactic.l.value-360
#b = cpos_src_obj_heal.galactic.b.value 

#hp.projscatter(l,b,lonlat=True,color='m')

ModuleNotFoundError: No module named 'healpy'

## 2 - CASDA RM Data